In [ ]:
import numpy as np
import timeit

import matplotlib.pyplot as plt

import torch
import torch.optim as optim

import fitsio 

import sys
sys.path.insert(0, './../')
import sdss_psf
import simulated_datasets_lib
import starnet_vae_lib
import sdss_dataset_lib
import plotting_utils
import image_statistics_lib
import utils
import psf_transform_lib2

import inv_kl_objective_lib as inv_kl_lib

import image_utils

import time

import json

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('torch version: ', torch.__version__)

from copy import deepcopy

In [ ]:
np.random.seed(453)
_ = torch.manual_seed(786)

# Data parameters

In [ ]:
# data parameters
with open('../data/default_star_parameters.json', 'r') as fp:
    data_params = json.load(fp)

In [ ]:
data_params

# The PSF

In [ ]:
# psf_dir = '../data/'
# psf_r = fitsio.FITS(psf_dir + 'sdss-002583-2-0136-psf-r.fits')[0].read()
# psf_i = fitsio.FITS(psf_dir + 'sdss-002583-2-0136-psf-i.fits')[0].read()
# psf_og = np.array([psf_r, psf_i])

bands = [2, 3]
psfield_file = '../../celeste_net/sdss_stage_dir/2583/2/136/psField-002583-2-0136.fit'
init_psf_params = torch.zeros(len(bands), 6)
for i in range(len(bands)):
    init_psf_params[i] = psf_transform_lib2.get_psf_params(
                                    psfield_file,
                                    band = bands[i])
power_law_psf = psf_transform_lib2.PowerLawPSF(init_psf_params.to(device))
psf_og = power_law_psf.forward().detach()


n_bands = psf_og.shape[0]

# Draw data

In [ ]:
background = torch.ones(psf_og.shape[0],
                        data_params['slen'],
                        data_params['slen']) * \
                torch.Tensor([686., 1123.])[:, None, None]

In [ ]:
# Draw from the same distribution I used in the sleep phase
n_images = 1

simulated_dataset = \
    simulated_datasets_lib.load_dataset_from_params(psf_og,
                    data_params,
                    background = background,
                    n_images = n_images,
                    transpose_psf = False, 
                    add_noise = True)

images_full = simulated_dataset.images.detach()
backgrounds_full = simulated_dataset.background.detach()
        
which_on = (simulated_dataset.fluxes > 0).any(2).squeeze()
        
true_full_locs = simulated_dataset.locs[:, which_on, :]
true_full_fluxes = simulated_dataset.fluxes[:, which_on, :]
        
simulator = simulated_dataset.simulator

In [ ]:
true_full_fluxes.shape

In [ ]:
true_full_locs.shape

In [ ]:
# histogram of fluxes
for i in range(psf_og.shape[0]): 
    plt.hist(np.log10(true_full_fluxes[:, :, i].numpy().flatten()), bins = 50);

In [ ]:
if n_bands > 1: 
    # histogram of colors
    colors = -2.5 * np.log10(true_full_fluxes[:, :, 1].numpy().flatten() / \
                      true_full_fluxes[:, :, 0].numpy().flatten())
    plt.hist(colors, bins = 50);

In [ ]:
# color x flux
if n_bands > 1: 
    plt.scatter(colors, np.log10(true_full_fluxes[:, :, 0].numpy().flatten()), marker = 'x', alpha = 0.5)

In [ ]:
images_full.min()

In [ ]:
true_full_fluxes.min()

# Load VAE

In [ ]:
optimize_fluxes = True

In [ ]:
star_encoder = starnet_vae_lib.StarEncoder(full_slen = data_params['slen'],
                                            stamp_slen = 7,
                                            step = 2,
                                            edge_padding = 2, 
                                            n_bands = n_bands,
                                            max_detections = 2, 
                                          estimate_flux_var = not(optimize_fluxes))

In [ ]:
star_encoder.load_state_dict(torch.load('../fits/results_2020-02-04/starnet_ri_sdss_params',
                               map_location=lambda storage, loc: storage))
star_encoder.eval(); 

In [ ]:
# check loss 
loss, counter_loss, locs_loss, fluxes_loss, perm_indx = \
    inv_kl_lib.get_encoder_loss(star_encoder, images_full, backgrounds_full, 
                                true_full_locs, true_full_fluxes)[0:5]

In [ ]:
print('loss: {:06f}'.format(loss))

In [ ]:
print(counter_loss.mean())
print(locs_loss.mean())
print(fluxes_loss.mean())

# Get image stamps

In [ ]:
# get image stamps
image_stamps, true_subimage_locs, true_subimage_fluxes, \
    true_subimage_n_stars, true_is_on_array = \
        star_encoder.get_image_stamps(images_full, true_full_locs, true_full_fluxes, 
                                      trim_images = False, clip_max_stars = True)
    
background_stamps = star_encoder.get_image_stamps(backgrounds_full, None, None, 
                                      trim_images = False)[0]

In [ ]:
foo = plt.hist(true_subimage_n_stars, bins=np.arange(max(true_subimage_n_stars) + 2))[0]
# plt.plot(foo[0] / star_encoder.weights, 'x')

In [ ]:
plt.hist(locs_loss.detach()[locs_loss != 0], bins = 100);


# get inferred parameters on stamps

In [ ]:
# Note that these variational parameters are estimated using the true number of stars!
stamp_loc_mean, stamp_loc_log_var, \
    stamp_log_flux_mean, stamp_log_flux_log_var, stamp_log_probs = \
        star_encoder(image_stamps, background_stamps, true_subimage_n_stars)

In [ ]:
map_n_stars_stamps = torch.argmax(stamp_log_probs, dim = 1).detach()

In [ ]:
(map_n_stars_stamps == true_subimage_n_stars).float().mean()

In [ ]:
plt.hist(map_n_stars_stamps, bins = np.arange(star_encoder.max_detections + 2))
plt.hist(true_subimage_n_stars, bins = np.arange(star_encoder.max_detections + 2), alpha = 0.2)

In [ ]:
from itertools import permutations

In [ ]:
perm_list = []
for perm in permutations(range(star_encoder.max_detections)):
    perm_list.append(perm)

In [ ]:
perm = np.zeros((image_stamps.shape[0], star_encoder.max_detections))
for i in range(image_stamps.shape[0]): 
    perm[i, :] = perm_list[perm_indx[i]]

### check parameters

In [ ]:
# permute true parameters 
def permute_params(locs, fluxes, perm): 
    batchsize = perm.shape[0]
    max_stars = perm.shape[1]
    
    n_bands = fluxes.shape[-1]

    locs_perm = torch.zeros((batchsize, max_stars, 2))
    fluxes_perm = torch.zeros((batchsize, max_stars, n_bands))
    seq_tensor = torch.LongTensor([i for i in range(batchsize)])

    for i in range(max_stars):
        locs_perm[:, i, :] = locs[seq_tensor, perm[:, i], :]
        fluxes_perm[:, i, :] = fluxes[seq_tensor, perm[:, i], :]
        
    return locs_perm, fluxes_perm

In [ ]:
locs_perm, log_fluxes_perm = permute_params(stamp_loc_mean, stamp_log_flux_mean, perm)
log_var_locs_perm, log_var_log_fluxes_perm = \
    permute_params(stamp_loc_log_var, stamp_log_flux_log_var, perm)

In [ ]:
map_subimage_locs = (locs_perm * \
                     true_is_on_array.unsqueeze(2).float()).detach()
map_subimage_fluxes = \
    (torch.exp(log_fluxes_perm) * \
                           true_is_on_array.unsqueeze(2).float()).detach()

In [ ]:
plt.plot(map_subimage_locs.flatten()[map_subimage_locs.flatten() > 0].detach(), 
         true_subimage_locs.flatten()[true_subimage_locs.flatten() > 0], '+')

plt.plot(map_subimage_locs.flatten()[map_subimage_locs.flatten() > 0].detach(), 
         map_subimage_locs.flatten()[map_subimage_locs.flatten() > 0].detach(), '-')

plt.xlabel('estimated')
plt.ylabel('truth')

In [ ]:
plt.plot(torch.log(true_subimage_fluxes.flatten()[true_subimage_fluxes.flatten() > 0]), 
         torch.log(map_subimage_fluxes.flatten()[map_subimage_fluxes.flatten() > 0].detach()), '+')

plt.plot(torch.log(map_subimage_fluxes.flatten()[map_subimage_fluxes.flatten() > 0].detach()), 
         torch.log(map_subimage_fluxes.flatten()[map_subimage_fluxes.flatten() > 0].detach()), '-')

plt.xlabel('truth')
plt.ylabel('estimated')

In [ ]:
est = log_fluxes_perm[true_subimage_fluxes > 0]
truth = torch.log(true_subimage_fluxes[true_subimage_fluxes > 0])

est_sd = torch.exp(0.5 * log_var_log_fluxes_perm[true_subimage_fluxes > 0])

zscore = (est - truth) / est_sd

plt.hist(zscore.detach(), bins = 50); 

In [ ]:
est = locs_perm[true_subimage_locs > 0]
truth = true_subimage_locs[true_subimage_locs > 0]

est_sd = torch.exp(0.5 * log_var_locs_perm[true_subimage_locs > 0])

zscore = (est - truth) / est_sd

plt.hist(zscore.detach(), bins = 50); 

In [ ]:
zscore.mean()

In [ ]:
zscore.var()

In [ ]:
# Not just fluxes, but also color?

if n_bands > 1: 
    map_color = torch.log10(map_subimage_fluxes[:, :, 1].flatten()[map_subimage_fluxes[:, :, 1].flatten() > 0] / \
                            map_subimage_fluxes[:, :, 0].flatten()[map_subimage_fluxes[:, :, 0].flatten() > 0])

    true_color = torch.log10(true_subimage_fluxes[:, :, 1].flatten()[true_subimage_fluxes[:, :, 1].flatten() > 0] / \
                            true_subimage_fluxes[:, :, 0].flatten()[true_subimage_fluxes[:, :, 0].flatten() > 0])

    plt.plot(map_color, true_color, 'x')
    plt.plot(map_color, map_color, '-')

In [ ]:

if n_bands > 1: 
    fig, axarr = plt.subplots(1, 2, figsize=(7, 4))

    axarr[0].scatter(true_color, 
                     true_subimage_fluxes[:, :, 1].flatten()[true_subimage_fluxes[:, :, 1].flatten() > 0])

    axarr[0].scatter(map_color, 
                     map_subimage_fluxes[:, :, 1].flatten()[true_subimage_fluxes[:, :, 1].flatten() > 0], )

# Check reconstructions 

In [ ]:
use_true_n_stars = False
if use_true_n_stars: 
    _n_stars = true_subimage_n_stars
else: 
    _n_stars = None

# get parameters on the full image 
map_locs_full_image, map_fluxes_full_image, map_n_stars_full = \
    star_encoder.sample_star_encoder(images_full, backgrounds_full, 
                                     return_map_n_stars = True,
                                     return_map_star_params = True, 
                                     n_stars = _n_stars)[0:3]
    
if _n_stars is not None: 
    assert map_n_stars_full == torch.sum(_n_stars)

assert map_n_stars_full == map_locs_full_image.shape[1]
assert map_n_stars_full == map_fluxes_full_image.shape[1]

In [ ]:
import flux_utils 

In [ ]:
if optimize_fluxes: 
    flux_estimator = flux_utils.EstimateFluxes(images_full, map_locs_full_image, map_n_stars_full,
                                            simulator.psf, backgrounds_full, pad = 5, 
                                            init_fluxes = map_fluxes_full_image)
    
    print(flux_estimator.get_loss())
    flux_estimator.optimize()
    print(flux_estimator.get_loss())
    map_fluxes_full_image = flux_estimator.return_fluxes()

In [ ]:
foo = plt.hist(torch.log10(true_full_fluxes).flatten())[1]; 
plt.hist(torch.log10(map_fluxes_full_image.flatten()), bins = foo, alpha = 0.5)

In [ ]:
# get reconstructed mean
vae_recon_mean = simulator.draw_image_from_params(locs = map_locs_full_image, 
                                                fluxes = map_fluxes_full_image,
                                                 n_stars = map_n_stars_full, 
                                                 add_noise = False).detach()

In [ ]:
band = 0

In [ ]:
fig, axarr = plt.subplots(1, 3, figsize=(15, 4))
im0 = axarr[0].matshow(images_full[0, band][5:95, 5:95])
fig.colorbar(im0, ax = axarr[0])

im1 = axarr[1].matshow(vae_recon_mean[0, band][5:95, 5:95])
fig.colorbar(im1, ax = axarr[1])

residual = torch.log10(vae_recon_mean[0, band]) - torch.log10(images_full[0, band])
_residual = (residual * 2.5)[5:95, 5:95]
# (torch.log(vae_recon_mean.squeeze()) - torch.log(images_full.squeeze()))[10:90, 10:90]
vmax = _residual.abs().max()
im2 = axarr[2].matshow(_residual, vmax = vmax, vmin = -vmax, cmap=plt.get_cmap('bwr'))
fig.colorbar(im2, ax = axarr[2])

In [ ]:
np.argwhere(_residual.numpy() < -.9)

In [ ]:
plt.hist(_residual.flatten())

In [ ]:
plt.matshow(_residual[40:50, 10:20]); 
plt.colorbar()

In [ ]:
def get_which_tile(x0, x1, tile_coords, edge_padding, stamp_slen): 
    coords = tile_coords + edge_padding
    
    view_slen = stamp_slen - 2 * edge_padding
    
    indx = torch.where((x0 > coords[:, 0]) & \
                       (x0 < coords[:, 0] + view_slen) & \
                       (x1 > coords[:, 1]) & \
                       (x1 < coords[:, 1] + view_slen))
        
    return tile_coords[indx], indx

In [ ]:
get_which_tile(24, 79, star_encoder.tile_coords, star_encoder.edge_padding, star_encoder.stamp_slen)

# check image patches

In [ ]:
f, axarr = plt.subplots(1, 3, figsize=(16, 4))

indx = 518 # int(np.random.choice(image_stamps.shape[0], 1))
# indx = int(np.random.choice(torch.where(true_subimage_n_stars == 2)[0].numpy(), 1))

plotting_utils.plot_subimage(axarr[0], images_full[0, band],
                            map_locs_full_image.squeeze(), 
                            true_full_locs.squeeze(), 
                            int(star_encoder.tile_coords[indx, 0]), 
                            int(star_encoder.tile_coords[indx, 1]), 
                            subimage_slen = star_encoder.stamp_slen, 
                            add_colorbar = True, 
                            global_fig = f)

plotting_utils.plot_subimage(axarr[1], vae_recon_mean[0, band],
                            map_locs_full_image.squeeze(), 
                            None, 
                            int(star_encoder.tile_coords[indx, 0]), 
                            int(star_encoder.tile_coords[indx, 1]), 
                            subimage_slen = star_encoder.stamp_slen, 
                            add_colorbar = True, 
                            global_fig = f)

foo = torch.log10(vae_recon_mean[0, band]) - torch.log10(images_full[0, band])
plotting_utils.plot_subimage(axarr[2], foo * 2.5, 
                            map_locs_full_image.squeeze(), 
                            None, 
                            int(star_encoder.tile_coords[indx, 0]), 
                            int(star_encoder.tile_coords[indx, 1]), 
                            subimage_slen = star_encoder.stamp_slen, 
                            add_colorbar = True, 
                            global_fig = f, 
                            diverging_cmap = True)

axarr[0].axvline(x=2, color = 'r')
axarr[0].axvline(x=4, color = 'r')
axarr[0].axhline(y=2, color = 'r')
axarr[0].axhline(y=4, color = 'r')

axarr[1].axvline(x=2, color = 'r')
axarr[1].axvline(x=4, color = 'r')
axarr[1].axhline(y=2, color = 'r')
axarr[1].axhline(y=4, color = 'r')

# Check out some summary statistics 

In [ ]:
n_elect_per_nmgy = 856.

In [ ]:
# completeness and tpr 
completeness, tpr, completeness_bool, tpr_bool = \
    image_statistics_lib.get_summary_stats(map_locs_full_image.squeeze(), 
                                           true_full_locs.squeeze(), 
                                           star_encoder.full_slen, 
                                           map_fluxes_full_image.squeeze(0)[:, 0], 
                                           true_full_fluxes.squeeze(0)[:, 0], 
                                          n_elect_per_nmgy)
    
print('completeness: {:0.3f}'.format(completeness))
print('true positive rate: {:0.3f}'.format(tpr))

In [ ]:
est_locs, est_fluxes = image_statistics_lib.filter_params(map_locs_full_image.squeeze(), 
                                     map_fluxes_full_image.squeeze(0)[:, 0], 101, 5)

est_mags = image_statistics_lib.convert_nmgy_to_mag(est_fluxes / n_elect_per_nmgy)

In [ ]:
est_locs[(tpr_bool == 0) & (est_mags < 16)]

In [ ]:
completeness_vec, mag_vec = \
    image_statistics_lib.get_completeness_vec(map_locs_full_image.squeeze(), 
                                           true_full_locs.squeeze(), 
                                           star_encoder.full_slen, 
                                           map_fluxes_full_image.squeeze(0)[:, 0], 
                                           true_full_fluxes.squeeze(0)[:, 0], 
                                             n_elect_per_nmgy)[0:2]

plt.plot(mag_vec[0:-1], completeness_vec, '--x')
plt.xlabel('true log flux')
plt.ylabel('completeness')

In [ ]:
tpr_vec, mag_vec = \
    image_statistics_lib.get_tpr_vec(map_locs_full_image.squeeze(), 
                                        true_full_locs.squeeze(), 
                                        star_encoder.full_slen, 
                                        map_fluxes_full_image.squeeze(0)[:, 0], 
                                        true_full_fluxes.squeeze(0)[:, 0], 
                                        n_elect_per_nmgy)[0:2]

plt.plot(mag_vec[0:-1], tpr_vec, '--x')
plt.xlabel('estimated log flux')
plt.ylabel('tpr')

# Posterior samples

In [ ]:
n_samples = 5

In [ ]:
sampled_locs_full_image, sampled_fluxes_full_image, sampled_n_stars_full = \
    star_encoder.sample_star_encoder(images_full, backgrounds_full, 
                                     n_stars = None, 
                                    return_map_n_stars = False,
                                    return_map_star_params = False, 
                                    n_samples = n_samples)[0:3]

In [ ]:
backgrounds_full.mean()

In [ ]:
sampled_n_stars_full

In [ ]:
if optimize_fluxes: 
    flux_estimator = flux_utils.EstimateFluxes(images_full, sampled_locs_full_image, sampled_n_stars_full,
                                            simulator.psf, sky_intensity, pad = 5, 
                                            init_fluxes = sampled_fluxes_full_image)
    
    print(flux_estimator.get_loss())
    flux_estimator.optimize()
    print(flux_estimator.get_loss())
    sampled_fluxes_full_image = flux_estimator.return_fluxes()

In [ ]:
completeness_sampled = torch.zeros(n_samples)
tpr_sampled = torch.zeros(n_samples)
for i in range(n_samples): 
    # completeness and tpr
    n_stars_i = sampled_n_stars_full[i]
    completeness_sampled[i], tpr_sampled[i] = \
        image_statistics_lib.get_summary_stats(sampled_locs_full_image[i][0:n_stars_i], 
                                               true_full_locs.squeeze(), 
                                               star_encoder.full_slen, 
                                               sampled_fluxes_full_image[i][0:n_stars_i, 0], 
                                               true_full_fluxes.squeeze(0)[:, 0], 
                                              n_elect_per_nmgy)[0:2]

print('completeness: {:0.3f}'.format(completeness_sampled.mean()))
print('true positive rate: {:0.3f}'.format(tpr_sampled.mean()))

In [ ]:
# get reconstructed mean
recon_sampled = simulator.draw_image_from_params(locs = sampled_locs_full_image, 
                                                fluxes = sampled_fluxes_full_image,
                                                 n_stars = sampled_n_stars_full, 
                                                 add_noise = False).detach()

In [ ]:
recon_sampled.shape

In [ ]:
for i in range(n_samples): 
    fig, axarr = plt.subplots(1, 3, figsize=(15, 4))
    im0 = axarr[0].matshow(images_full[0, band][5:95, 5:95])
    fig.colorbar(im0, ax = axarr[0])

    im1 = axarr[1].matshow(recon_sampled[i, band][5:95, 5:95])
    fig.colorbar(im1, ax = axarr[1])

    residual = torch.log10(recon_sampled[i, band]) - torch.log10(images_full[0, band])
    _residual = (residual * 2.5)[5:95, 5:95]
    # (torch.log(vae_recon_mean.squeeze()) - torch.log(images_full.squeeze()))[10:90, 10:90]
    vmax = _residual.abs().max()
    im2 = axarr[2].matshow(_residual, vmax = vmax, vmin = -vmax, cmap=plt.get_cmap('bwr'))
    fig.colorbar(im2, ax = axarr[2])

In [ ]:
np.argwhere(_residual.numpy() < -2)

In [ ]:
get_which_tile(63, 89, star_encoder.tile_coords, star_encoder.edge_padding, star_encoder.stamp_slen)

In [ ]:
indx = 1483 # int(np.random.choice(image_stamps.shape[0], 1))

f, axarr = plt.subplots(1, 3, figsize=(16, 4))
plotting_utils.plot_subimage(axarr[0], images_full[0, band],
                            map_locs_full_image.squeeze(), 
                            true_full_locs.squeeze(), 
                            int(star_encoder.tile_coords[indx, 0]), 
                            int(star_encoder.tile_coords[indx, 1]), 
                            subimage_slen = star_encoder.stamp_slen, 
                            add_colorbar = True, 
                            global_fig = f)

plotting_utils.plot_subimage(axarr[1], vae_recon_mean[0, band],
                            map_locs_full_image.squeeze(), 
                            None, 
                            int(star_encoder.tile_coords[indx, 0]), 
                            int(star_encoder.tile_coords[indx, 1]), 
                            subimage_slen = star_encoder.stamp_slen, 
                            add_colorbar = True, 
                            global_fig = f)

foo = torch.log10(vae_recon_mean[0, band] / images_full[0, band])
plotting_utils.plot_subimage(axarr[2], foo, 
                            map_locs_full_image.squeeze(), 
                            None, 
                            int(star_encoder.tile_coords[indx, 0]), 
                            int(star_encoder.tile_coords[indx, 1]), 
                            subimage_slen = star_encoder.stamp_slen, 
                            add_colorbar = True, 
                            global_fig = f, 
                            diverging_cmap = True)

axarr[0].axvline(x=2, color = 'r')
axarr[0].axvline(x=4, color = 'r')
axarr[0].axhline(y=2, color = 'r')
axarr[0].axhline(y=4, color = 'r')

axarr[1].axvline(x=2, color = 'r')
axarr[1].axvline(x=4, color = 'r')
axarr[1].axhline(y=2, color = 'r')
axarr[1].axhline(y=4, color = 'r')

for i in range(n_samples): 
    f, axarr = plt.subplots(1, 3, figsize=(16, 4))

    plotting_utils.plot_subimage(axarr[0], images_full[0, band],
                                sampled_locs_full_image[i], 
                                true_full_locs.squeeze(), 
                                int(star_encoder.tile_coords[indx, 0]), 
                                int(star_encoder.tile_coords[indx, 1]), 
                                subimage_slen = star_encoder.stamp_slen, 
                                add_colorbar = True, 
                                global_fig = f)

    plotting_utils.plot_subimage(axarr[1], recon_sampled[i, band],
                                sampled_locs_full_image[i], 
                                None, 
                                int(star_encoder.tile_coords[indx, 0]), 
                                int(star_encoder.tile_coords[indx, 1]), 
                                subimage_slen = star_encoder.stamp_slen, 
                                add_colorbar = True, 
                                global_fig = f)

    foo = torch.log10(recon_sampled[i, band] / images_full[0, band])
    plotting_utils.plot_subimage(axarr[2], foo, 
                                sampled_locs_full_image[i], 
                                None, 
                                int(star_encoder.tile_coords[indx, 0]), 
                                int(star_encoder.tile_coords[indx, 1]), 
                                subimage_slen = star_encoder.stamp_slen, 
                                add_colorbar = True, 
                                global_fig = f, 
                                diverging_cmap = True)

    axarr[0].axvline(x=2, color = 'r')
    axarr[0].axvline(x=4, color = 'r')
    axarr[0].axhline(y=2, color = 'r')
    axarr[0].axhline(y=4, color = 'r')

    axarr[1].axvline(x=2, color = 'r')
    axarr[1].axvline(x=4, color = 'r')
    axarr[1].axhline(y=2, color = 'r')
    axarr[1].axhline(y=4, color = 'r')